In [25]:
import csv
import pandas as pd
import datetime

items = pd.read_csv('LAKEHOUSE_SILVER_LAYER/items.csv')
prod = pd.read_csv('LAKEHOUSE_SILVER_LAYER/production.csv')
warehouse = pd.read_csv('LAKEHOUSE_SILVER_LAYER/warehouse.csv')

In [26]:
modwarehouse = warehouse

In [27]:
modwarehouse["datetime"] = modwarehouse["Registering Date"].apply(lambda x: datetime.datetime.strptime(x, "%d-%m-%Y"))
modwarehouse

,Entry No_,Journal Batch Name,Line No_,Registering Date,Location Code,Zone Code,Bin Code,Description,Item No_,Quantity,...,Qty_ per Unit of Measure,Unit of Measure Code,Serial No_,Lot No_,Warranty Date,Expiration Date,Phys Invt Counting Period Code,Phys Invt Counting Period Type,Dedicated,datetime
0,1,Unnamed: 1,10000,30-11-2022,SEZ3,TMG,TMG.1,STATIC SHIELDED BAGS OF SIZE 280x220,135677,175,...,1.5,EA,Unnamed: 34,IDS-21E2-00560,30-05-2026 13:49,08-11-2021 00:00,PC_001,1.6,1.7,2022-11-30
1,2,NaN,10000,18-12-2022,SEZ2,IGS,IGS,STATIC SHIELDED BAGS OF SIZE 280x220,119852,175,...,1.0,EA,NaN,IDS-21E2-00125,21-02-2027 17:23,08-11-2021 00:00,PC_002,1.0,1.0,2022-12-18
2,3,NaN,10000,30-05-2021,SEZ2,IGS,IGS,"RoHS,Label, 4X6 inch",147831,0,...,1.0,EA,NaN,IDS-21E2-00324,06-07-2025 08:00,08-11-2021 00:00,PC_003,1.0,1.0,2021-05-30
3,4,NaN,10000,13-04-2021,SEZ2,IGS,IGS,"RoHSLabel, 3.5 X 1 inch blank (outer box labels/",105695,0,...,1.0,EA,NaN,IDS-21E2-00564,20-06-2025 06:00,14-05-2026 00:03,PC_004,1.0,1.0,2021-04-13
4,5,NaN,10000,26-12-2021,SEZ3,TMG,TMG,"RoHS,Security Tape",141663,175,...,1.0,EA,NaN,IDS-21E2-00039,13-11-2025 06:38,06-05-2026 01:11,PC_005,1.0,1.0,2021-12-26
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11995,11996,NaN,10000,26-06-2021,SEZ3,TMG,TMG,"RoHS,LABEL-CRUCIAL LABEL--*-54*12.8MM-GLOSS",119852,175,...,1.0,EA,NaN,IDS-21E2-00612,14-08-2026 21:30,27-10-2025 04:07,PC_003,1.0,1.0,2021-06-26
11996,11997,NaN,10000,17-02-2022,SEZ3,TMG,TMG,"RoHS PCB BOARD, IQ CONTROLLER",147359,175,...,1.0,EA,NaN,IDS-21E2-00039,03-04-2025 17:41,08-11-2021 00:00,PC_005,1.0,1.0,2022-02-17
11997,11998,NaN,10000,27-04-2021,SEZ3,TMG,TMG,_RARE_,112048,0,...,1.0,EA,NaN,IDS-21E2-00125,11-10-2027 06:40,30-07-2027 04:49,PC_004,1.0,1.0,2021-04-27
11998,11999,NaN,10000,17-03-2022,SEZ3,TMG,TMG,OTCT-377*192*309MM-200PCS DIMM-Y,119852,0,...,1.0,EA,NaN,IDS-21E2-00564,23-11-2026 13:42,06-08-2027 04:00,PC_004,1.0,1.0,2022-03-17


In [28]:
df = modwarehouse.groupby(["Lot No_", "Bin Code", "Item No_"])["datetime"].min().reset_index(name="Min_Registering_Date")
df["Min_Registering_Date"] = df["Min_Registering_Date"].apply(lambda x: x.strftime("%d-%m-%Y"))

#df["Min_Registering_Date"] = modwarehouse[modwarehouse["Lot No_"]]["Registering Date"].min()
#df.columns = ["Min_Registering_Date"]

#df = df[df["Quantity"]>0]
#df

In [29]:
df = df.set_index(["Lot No_", "Bin Code", "Item No_"])
df["Quantity"] = list(modwarehouse.groupby(["Lot No_", "Bin Code", "Item No_"])["Quantity"].sum())

In [30]:
def func(lis):
    return sorted(lis.unique())[0]
df["Zone Code"] = list(modwarehouse.groupby(["Lot No_", "Bin Code", "Item No_"])["Zone Code"].apply(lambda a: sorted(a.unique())[0]))

In [31]:
#df.get_group("IDS-21E2-00028", "IGS")#,"104754")
#modwarehouse.groupby(["Lot No_", "Bin Code", "Item No_"])
df


Min_Registering_Date  Quantity Zone Code
Lot No_        Bin Code Item No_                                         
IDS-21E2-00028 IGS      1000457            19-05-2021      1000       IGS
                        102078             08-08-2022       175       IGS
                        102565             28-01-2022       350       IGS
                        102671             13-12-2021       200       IGS
                        104754             24-02-2021         0       IGS
...                                               ...       ...       ...
IDS-21E2-01221 IGS      108259             02-12-2021       250       IGS
IDS-21E2-01233 TMG      125948             19-03-2022       175       TMG
IDS-21E2-01236 IGS      147831             16-08-2022       175       IGS
IDS-21E2-01268 TMG      111742             08-04-2022         0       TMG
                        141305             23-02-2021         0       TMG

[4925 rows x 3 columns]

In [32]:
#df["Days from Registration"] = list(modwarehouse.groupby(["Lot No_", "Bin Code", "Item No_"])["datetime"].min().apply(lambda a: datetime.datetime.now() - a).apply(lambda a: a.days))
df["Days from Registration"] = list(modwarehouse.groupby(["Lot No_", "Bin Code", "Item No_"])["datetime"].min())
df["Days from Registration"] = datetime.datetime.now() - df["Days from Registration"]
df["Days from Registration"] = df["Days from Registration"].apply(lambda a: a.days)

In [33]:
df = df[df["Quantity"]>0]


In [34]:
df

Min_Registering_Date  Quantity Zone Code  \
Lot No_        Bin Code Item No_                                            
IDS-21E2-00028 IGS      1000457            19-05-2021      1000       IGS   
                        102078             08-08-2022       175       IGS   
                        102565             28-01-2022       350       IGS   
                        102671             13-12-2021       200       IGS   
                        105581             15-05-2021      1175       IGS   
...                                               ...       ...       ...   
IDS-21E2-01217 TMG      122402             07-12-2022       200       TMG   
                        123546             24-08-2021       200       TMG   
IDS-21E2-01221 IGS      108259             02-12-2021       250       IGS   
IDS-21E2-01233 TMG      125948             19-03-2022       175       TMG   
IDS-21E2-01236 IGS      147831             16-08-2022       175       IGS   

                                  Days from Registration  
Lot No_        Bin Code Item No_                          
IDS-21E2-00028 IGS      1000457                      601  
                        102078                       155  
                        102565                       347  
                        102671                       393  
                        105581                       605  
...                                                  ...  
IDS-21E2-01217 TMG      122402                        34  
                        123546                       504  
IDS-21E2-01221 IGS      108259                       404  
IDS-21E2-01233 TMG      125948                       297  
IDS-21E2-01236 IGS      147831                       147  

[3994 rows x 4 columns]

In [35]:
df.loc["IDS-21E2-00028","IGS","102671"]

Min_Registering_Date      13-12-2021
Quantity                         200
Zone Code                        IGS
Days from Registration           393
Name: (IDS-21E2-00028, IGS, 102671), dtype: object

In [36]:
df.to_csv("LAKEHOUSE_GOLD_LAYER/sc_step1.csv")